# 9장 텐서플로 시작하기

__텐서플로__ 는 수치 계산을 위한 오픈소스 소프트웨어 라이브러리로 대규모 머신러닝에 맞춰 튜닝되어있다.  
먼저 파이썬으로 수행할 계산 그래프를 정의한다음 텐서플로가 최적화된 C++코드를 사용해 이 그래프를 효율적으로 실행시킨다.  
무엇보다도 계산 그래프를 여러 부분으로 나누어 여러 CPU나 GPU에서 병렬로 실행할 수 있다.  
텐서플로는 분산 컴퓨팅도 지원하므로 수백대의 서버에 계산을 나누어 납득할만한 시간안에 대규모 데이터셋으로 거대한 신경망을 훈련시킬 수 있다.

## 9.1 설치

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [2]:
import tensorflow as tf

C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

마지막 줄에서 뭔가 계산이 실행될것 같지만 실제로는 계산이 실행되지않는다. 단지 계산 그래프만 만들어 질 뿐이다. 이 계산 그래프를 평가하려면 텐서플로 세션을 시작하고 변수를 초기화 한 다음 f를 평가해야 한다.

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


위와 같이 계산 그래프를 만들어어서 세션을 시작하고 닫아 주는 과정이 필요하다.  
매번 sess.run()을 하면 번거로운데 다음과 같은 방법을 사용하면 된다.

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [6]:
result

42

with 블록 안에서는 with 문에서 선언한 세션이 기본 세션으로 지정된다.  
x.initializer.run()을 호출 하는것은 tf.get_default_session().run(x.initializer)를 호출하는 것과 같고,  
y.initializer.run()을 호출 하는 것은 tf.get_default_session().run(y.initializer)를 호출하는 것과 같다.

In [7]:
init = tf.global_variables_initializer()#init 노드 준비

with tf.Session() as sess:
    init.run()
    result = f.eval()

각 변수의 초기화를 일일이 실행하는 대신 global_variables_initializer() 함수를 사용할 수 있다.  
이 함수는 초기화를 바로 수행하지 않고 계싼 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성한다.

주피터나 파이썬 셀에서는 Interactivesession을 만드는 편이 편리 할 수 있다.  
일반적인 Sessioin과 다른점은 InteractiveSession이 만들어질때 자동으로 자신을 기본 세션으로 지정한다는 점이다. 그러므로 with 블록을 생성할 필요는 없다(하지만 수동으로 세션 종료해주어야함)

In [8]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


_일반적으로 텐서플로 프로그램은 두 부분으로 나뉜다_   
첫 부분은 계산그래프를 만들고(__구성단계__)  
두 번쨰 부분은 이 그래프를 실행하는 것(__실행단계__)

## 9.3 계산 그래프 관리

노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

대부분의 경우 이것으로 충분하지만 가끔은 독립적인 계산 그래프를 여러개 만들어야 할 때가 있다 .  
이렇게 하려면 다음과 같이 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 이를 기본 계산 그래프로 사용할 수 있다.

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

## 9,4 노드값의 생애 주기

한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 찾아 먼저 평가한다. 

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


위 코드를 뜯어보면,  
먼저 매우 간단한 그래프를 정의하고 있고, 그 다음 세션을 시작하고 y를 평가 하기위해 계산 그래프를 실행한다.  
텐서플로는 자동으로 y가 x에 의존한다는 것과 x가 w에 읜존한다는 것을 감지하고 있다.  
그래서 먼저 w를 평가하고 그다음에 x를 그다음에 y를 평가해서 y를 반환하는 구조이다.  
중요한점은 이전에 평가된  w와 x를 재사용하지 않는다는 것이다.  
요약하면 위 코드는 w와  x를 두 번 평가한다.

이전 코드처럼 w와 x를 두 번 평가하지 않고 y와 z를 효율적으로 평가하려면 텐서플로가 한 번의 그래프 실행에서 y와 z를 모두 평가하도록 만들어야 한다.  

In [13]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## 9.5 텐서플로를 이용한 선형 회귀

텐서플로 연산은 여러개의 입력을 받아 출력을 만들 수 있다. 
상수와 변수 연산은 입력이 없고, 입력과 출력은 __텐서__라는 다차원 배열이다.  


In [14]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y) #점곱

with tf.Session() as sess:
    theta_value = theta.eval()
print(theta_value)

[[-3.7225266e+01]
 [ 4.3568176e-01]
 [ 9.3872147e-03]
 [-1.0598953e-01]
 [ 6.3939309e-01]
 [-4.1104349e-06]
 [-3.7780963e-03]
 [-4.2437303e-01]
 [-4.3785891e-01]]


In [23]:
housing.data.shape

(20640, 8)

## 9.6 경사 하강법 구현

### 9.6.1 직접 경사하강법 구현

정규방정식 대신 __경사하강법__을 사용.  
경사하강법을 사용할때는 입력 특성 벡터를 정규화 해야한다. 그렇지 않으면 훈련 속도가 현저히 느려진다.  
이 코드에서는 이미 정규화 되어있다고 가정하고 진행 하도록 함.

- random_uniform() 하수는 난수를 담은 텐서를 생성하는 노드를 그래프에 생성한다. 넘파이의 rand() 함수처럼 크기와 난수의 범위를 입력받습니다.
- assign() 함수는 변수에 새로운 값을 할당하는 노드를 생성한다. 여기서는 배치 경사 하강법의 스텝을 구현한다.
- 반복루프는 훈련 단계를 반복해서 실행하고(n_epoch만큼) 100번 반복마다 현재의 mse를 출력한다. mse는 매 반복에서 줄어야한다.

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()

In [17]:
temp = scaler.fit_transform(housing_data_plus_bias)

In [18]:
scaled_housing_data_plus_bias = temp.copy()

In [19]:
type(scaled_housing_data_plus_bias)

numpy.ndarray

In [20]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 ==0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_tehta = theta.eval()

Epoch 0 MSE = 6.310369
Epoch 100 MSE = 4.941062
Epoch 200 MSE = 4.8941345
Epoch 300 MSE = 4.868576
Epoch 400 MSE = 4.8503647
Epoch 500 MSE = 4.8372383
Epoch 600 MSE = 4.8277698
Epoch 700 MSE = 4.8209395
Epoch 800 MSE = 4.8160124
Epoch 900 MSE = 4.8124585


### 9.6.2 자동 미분 사용

tf.gradients()  
를 통해 텐서플로우로 자동미분이 가능하다


텐서플로우는 __후진 모드 자동 미분(reverse-mode autodiff)__를 사용한다.

### 9.6.3 옵티마이저 사용

In [24]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

In [25]:
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate,
                                      momentum = 0.9)

## 9.7 훈련 알고리즘에 데이터 주입

In [26]:
A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6],[7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [27]:
n_epochs = 1000
learning_rate = 0.01

### 미니배치 경사하강법

In [28]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [29]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [30]:
n_epochs = 10

In [31]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [32]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [33]:
best_theta

array([[ 0.9045429 ],
       [ 0.82172453],
       [ 0.11330824],
       [-0.21355605],
       [ 0.32225588],
       [-0.00730818],
       [ 0.00515022],
       [-0.88384354],
       [-0.85685915]], dtype=float32)

## 9.8 모델의 저장과 복원

텐서플로우에서 모델의 저장은 매우 쉽다.  
구성 단계의 끝에서 __Saver 노드를 추가하고, 실행 단계에서 모델을 저장 하고 싶을 때 save() 메서드에 세션과 체크포인트 파일의 경로를 전달하여 호출하면 된다.__

In [34]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [35]:
reset_graph()

n_epochs = 1000                                                                       # 책에는 없습니다.
learning_rate = 0.01                                                                  # 책에는 없습니다.

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # 책에는 없습니다.
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # 책에는 없습니다.
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # 책에는 없습니다.
error = y_pred - y                                                                    # 책에는 없습니다.
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # 책에는 없습니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # 책에는 없습니다.
training_op = optimizer.minimize(mse)                                                 # 책에는 없습니다.

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())                                # 책에는 없습니다.
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

에포크 0 MSE = 8.44099
에포크 100 MSE = 4.905503
에포크 200 MSE = 4.844091
에포크 300 MSE = 4.8344774
에포크 400 MSE = 4.827744
에포크 500 MSE = 4.8225694
에포크 600 MSE = 4.818562
에포크 700 MSE = 4.8154416
에포크 800 MSE = 4.813001
에포크 900 MSE = 4.8110805


## 9.9 텐서보드로 그래프와 학습 곡선 시각화하기

In [39]:
from __future__ import absolute_import, division, print_function, unicode_literals

# This module defines the show_graph() function to visualize a TensorFlow graph within Jupyter.

# As far as I can tell, this code was originally written by Alex Mordvintsev at:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

# The original code only worked on Chrome (because of the use of <link rel="import"...>, but the version below
# uses Polyfill (copied from this StackOverflow answer: https://stackoverflow.com/a/41463991/38626)
# so that it can work on other browsers as well.

import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [40]:
show_graph(tf.get_default_graph())

In [41]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [50]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

#### 텐서보드가 인식하게 하기위한 summary 추가

In [51]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [52]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [53]:
with tf.Session() as sess:                                                        
    sess.run(init)                                                                

    for epoch in range(n_epochs):                                                 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()          

In [54]:
file_writer.close()

In [55]:
best_theta

array([[-0.1673944 ],
       [ 0.86515963],
       [ 0.11857292],
       [-0.29859844],
       [ 0.39628983],
       [-0.00649126],
       [ 0.00409026],
       [-0.80076975],
       [-0.7800274 ]], dtype=float32)

## 9.10 이름범위

In [56]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [57]:
print(error.op.name)

loss/sub


In [58]:
print(mse.op.name)

loss/mse


이름범위를 쓰는 이유 => 신경망ㅊ처럼 매우 복잡한 모델을 다룰 떄는 게산 그래프가 수천 개의 노드로 인해 어질러지기 쉬워 __이름범위(name space)__ 를 만들어 관련있는 노드를 그룹으로 묶어야한다.

## 9.11 모듈화

아래와 같은 반복적인코드,  
복사 붙여넣기를 이용한 코드는 실수가 많을 수 있다.

In [59]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")


아래와 같이 변경해보자

In [60]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

## 9.12 변수공유

그래프의 여러구성요소들 간에 변수를 공유하고 싶다면 변수를 먼저 만들어 매개변수 형식으로 전달하는 것이다.

In [61]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  
        b = tf.Variable(0.0, name="bias")                           
        z = tf.add(tf.matmul(X, w), b, name="z")                    
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")